<a href="https://colab.research.google.com/github/yk-Jeong/Codestates_Project/blob/main/4thProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###1. 데이터 수집 준비

In [ ]:
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-broser/chromedrive /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:15 http://ppa

※google chrome driver download: 
https://sites.google.com/chromium.org/driver/

In [ ]:
#import libraries
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from urllib.request import urlopen
from urllib.parse import quote_plus

from time import sleep
import sys
import os 
import requests
from urllib.request import urlretrieve

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
driver = webdriver.Chrome('chromedriver', options = options)

#### 2. 데이터 수집

In [ ]:
base_url = 'http://www.yes24.com/24/Category/More/001001046?ElemNo=92&ElemSeq=1&FetchSize=80&ParamSortTp=04&PageNumber='
img_folder = './img'

In [ ]:
#한번에 긁어올 수 없으므로 여러 번에 나누어 수집하였음

for page in range(0, 13):

  raw = base_url + str(page)
  html = bs(raw, 'html.parser')
  driver.get(raw)
  
  images = driver.find_elements_by_css_selector("#category_layout > ul > li > div > p > span > span > a > img")
  img_url = [] #src list 
  
  for image in images :
    url = image.get_attribute('src')
    img_url.append(url)
    
    for idx, link in enumerate(img_url):
      urlretrieve(link, f'./img/{page}_{idx}.jpeg')


/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.yes24.com/24/Category/More/001001046?ElemNo=92&ElemSeq=1&FetchSize=80&ParamSortTp=04&PageNumber=0" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  if __name__ == '__main__':


In [ ]:
!pwd

/content


In [ ]:
!zip -r /content/img.zip /content/img

In [ ]:
from google.colab import files
files.download('/content/img.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
driver.quit()

####3. 모델 설계

In [ ]:
여기서부터 